In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [ ]:
from YT_implementations import *

In [ ]:
# Data preprocessing

mean = np.mean(tX, axis=0)
std = np.std(tX, axis=0)

tX_train = standardization(tX, mean, std)

In [ ]:
seed = 7

### Least Squares GD

In [ ]:
max_iters = 500
gammas = np.linspace(0.001, 0.2, 20)

losses = []
accuracies = []

for gamma in gammas:
    initial_w = np.zeros(tX_train.shape[1])
    w_LSGD, loss_LSGD = least_squares_GD(y, tX_train, initial_w, max_iters, gamma)

    y_pred_LSGD = predict_labels(w_LSGD, tX_train)
    acc_LSGD = compute_accuracy(y_pred_LSGD, y)
    
    losses.append(loss_LSGD)
    accuracies.append(acc_LSGD)
    
gamma_opt = gammas[np.argmin(losses)]
print(gamma_opt, accuracies[np.argmin(losses)])

In [ ]:
plt.plot(gammas, losses)

In [ ]:
max_iters = 1000
gamma = gamma_opt

initial_w = np.zeros(tX_train.shape[1])
w_LSGD, loss_LSGD = least_squares_GD(y, tX_train, initial_w, max_iters, gamma)

y_pred_LSGD = predict_labels(w_LSGD, tX_train)
acc_LSGD = compute_accuracy(y_pred_LSGD, y)

acc_LSGD

### Least Squares SGD

In [ ]:
max_iters = 1000
gamma = gamma_opt

initial_w = np.zeros(tX_train.shape[1])
w_LSSGD, loss_LSSGD = least_squares_SGD(y, tX_train, initial_w, max_iters, gamma_SGD)

y_pred_LSSGD = predict_labels(w_LSSGD, tX_train)
acc_LSSGD = compute_accuracy(y_pred_LSSGD, y)

acc_LSSGD

### Least Squares

In [ ]:
# Opt Degree

k_fold = 5
k_indices = build_k_indices(y, k_fold, seed)

degrees = [1, 3, 5, 7, 9]

losses_opt = []

for degree in degrees:
    losses = []

    for k in range(k_fold):
        w, loss, acc_tr, acc_te = cross_validation(y, tX_train, k_indices, k, degree, False, least_squares)
        losses.append(loss)
            
    losses_opt.append(np.mean(losses))
    
degree_opt = degrees[np.argmin(losses_opt)]

print(degree_opt)

In [ ]:
plt.plot(degrees, losses_opt)

In [ ]:
tX_train_aug = build_poly(tX_train, degree_opt)

w_LS, loss_LS = least_squares(y, tX_train_aug)

y_pred_LS = predict_labels(w_LS, tX_train_aug)
acc_LS = compute_accuracy(y_pred_LS, y)

acc_LS

### Ridge Regression

In [ ]:
# Parameters Tuning

k_fold = 5
k_indices = build_k_indices(y, k_fold, seed)

degrees = [1, 3, 5, 7, 9]
#lambdas = np.logspace(-4, 0, 30)
lambdas = [0.05, 0.001]

losses_opt = []
lambdas_opt = []

for degree in degrees:
    losses = []
    
    for lambda_ in lambdas:
        losses_tmp = []

        for k in range(k_fold):
            w, loss_tr, loss_te, acc_tr, acc_te = cross_validation(y, tX_train, k_indices, k, degree, False, ridge_regression, lambda_=lambda_)
            losses_tmp.append(loss_te)
            
        losses.append(np.mean(losses_tmp))

    ind_lambda_opt = np.argmin(losses)
    lambdas_opt.append(lambdas[ind_lambda_opt])
    losses_opt.append(losses[ind_lambda_opt])
    
degree_opt = degrees[np.argmin(losses_opt)]
lambda_opt = lambdas_opt[np.argmin(losses_opt)]

print(degree_opt)
print(lambda_opt)
    

In [ ]:
plt.plot(degrees, losses_opt)

In [ ]:
tX_train_aug = build_poly(tX_train, degree_opt)

w_RR, loss_RR = ridge_regression(y, tX_train_aug, lambda_opt)

y_pred_RR = predict_labels(w_RR, tX_train_aug)
acc_RR = compute_accuracy(y_pred_RR, y)

acc_RR

### Logistic Regression

In [ ]:
y_logistic = y.copy()
y_logistic[y_logistic == -1] = 0

In [ ]:
# Parameters Tuning

max_iters = 100

k_fold = 5
k_indices = build_k_indices(y, k_fold, seed)

#gammas = np.logspace(-4, 1, 30)
gammas = [0.05, 0.001, 0.1]

losses_opt = []
gammas_opt = []
    
for gamma in gammas:
    losses = []
    for k in range(k_fold):
        w, loss_tr, loss_te, acc_tr, acc_te = cross_validation(y_logistic, tX_train, k_indices, k, degree, True, logistic_regression, initial_w=None, max_iters=max_iters, gamma=gamma)
        losses_tmp.append(loss_te)

    losses.append(np.mean(losses_tmp))

ind_gamma_opt = np.argmin(losses)
lambdas_opt.append(gammas[ind_lambda_opt])
losses_opt.append(losses[ind_lambda_opt])
    
gamma_opt = gammas_opt[np.argmin(losses_opt)]

print(gamma_opt)

In [ ]:
tX_train_aug = build_poly(tX_train, degree_opt=1)
initial_w = np.zeros(tX_train.shape[1])
w_LR, loss_LR = logistic_regression(y_logistic, tX_train, initial_w, max_iters, gamma_opt)

y_pred_LR = predict_labels(w_LR, x_train)
acc_LR = compute_accuracy(y_pred_LR, y)

print(loss_LR)

acc_LR

### Regularized Logistic Regression

In [ ]:
initial_w = np.zeros(x_train.shape[1])
w_RLR, loss_RLR = reg_logistic_regression(y_logistic, x_train, lambda_, initial_w, max_iters, gamma)

y_pred_RLR = predict_labels(w_RLR, x_train)
acc_RLR = compute_accuracy(y_pred_RLR, y)

acc_RLR

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv'
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
x_test = standardization(tX_test, mean, std)

In [ ]:
OUTPUT_PATH = 'output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)